In [8]:
base_dir = "/home3/skaasyap/willett"
batch_size=8
device = 'cuda:3'
sentence_path = '/home3/skaasyap/my_list.pkl'

model_paths = ['/home3/skaasyap/willett/outputs/consistency_0.2']
model_types = ['t'] #transformer or rnn for each path

In [9]:
import numpy as np
import neural_decoder.lm_utils as utils
from neural_decoder.load_models import loadModel, loadTransformerModel, getDatasetLoaders, rnn_forward, transformer_forward, run_ngram_model, convert_sentence
import pickle

In [10]:
with open(sentence_path, 'rb') as f:
    ground_truth_sentences = pickle.load(f)

In [ ]:
lmDir = base_dir+'/lm/languageModel'
ngramDecoder = utils.build_lm_decoder(
    lmDir,
    acoustic_scale=0.8, #1.2
    nbest=1,
    beam=18
)

In [ ]:
trainLoader, testLoader, loadedData = getDatasetLoaders(
    "/home3/skaasyap/willett/data_log_both",
    batch_size,
)

In [ ]:
for i, model_path in enumerate(model_paths):

    if(model_types[i] == 't'):
        model, args = loadTransformerModel(model_path)
        model = model.to(device)
        all_logits, trial_lengths, avgDayLoss, cer = transformer_forward(model, testLoader, device)
    else:
        model, args = loadModel(model_path)
        all_logits, trial_lengths, avgDayLoss, cer = transformer_forward(model, testLoader, device)

    
    all_logits = [l.cpu().numpy().astype('float32') for l in all_logits]
    trial_lengths = [l.cpu().numpy().astype('int') for l in trial_lengths]
    all_logits = [l for batch in all_logits for l in list(batch)]
    trial_lengths = [l for batch in trial_lengths for l in list(batch)]

    blank_penalty = np.log(2)
    llm_outputs = run_ngram_model(all_logits, trial_lengths, ngramDecoder, blank_penalty)

    print(len(llm_outputs), len(ground_truth_sentences))

    for i in range(len(ground_truth_sentences)):
        ground_truth_sentences[i] = ground_truth_sentences[i].strip()
        llm_outputs[i] = llm_outputs[i].strip()

    for i in range(len(ground_truth_sentences)):
        ground_truth_sentences[i] = convert_sentence(ground_truth_sentences[i])

    cer, wer = utils._cer_and_wer(llm_outputs, ground_truth_sentences, outputType='speech_sil', returnCI=True)

    print(f"cer: {cer}") # cer average, confidence interval 
    print(f"wer: {wer}") #wer average, confidence interval

/home3/skaasyap/miniconda3/envs/willett/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/augmentations.py:170: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return self.conv(input, weight=self.weight, groups=self.groups, padding="same")
/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor

ctc loss: 0.823501, cer: 0.138253
880 880
cer: (0.11273744234191697, 0.10408587030547085, 0.12155142019074096)
wer: (0.16730314602655028, 0.1546729334880064, 0.18015954606576218)
